In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression

# 기본 데이터 확인

In [2]:
df_2025 = pd.read_csv("./tmdb_v3_output/tmdb_2025.csv")
df_2025.head()

,title,release_date,type,runtime,season_count,episode_count,genre,director,cast,country,language,production_company,synopsis,tags,imdb_rating,imdb_review_count
0,Final Destination Bloodlines,2025-05-14,movie,109.0,NaN,NaN,"Horror, Mystery","Zach Lipovsky, Adam B. Stein","Kaitlyn Santa Juana, Teo Briones, Richard Harm...",US,en,"New Line Cinema, Practical Pictures, Freshman ...","Plagued by a violent recurring nightmare, coll...","restaurant, gore, sequel, premonition, fate, f...",7.166,826
1,The Amateur,2025-04-09,movie,123.0,NaN,NaN,"Thriller, Action",James Hawes,"Rami Malek, Holt McCallany, Danny Sapani, Rach...",US,en,"20th Century Studios, Hutch Parker Entertainme...",After his life is turned upside down when his ...,"central intelligence agency (cia), based on no...",6.907,523
2,STRAW,2025-06-05,movie,105.0,NaN,NaN,"Thriller, Drama, Crime",Tyler Perry,"Taraji P. Henson, Sherri Shepherd, Teyana Tayl...",US,en,Tyler Perry Studios,What will be her last straw? A devastatingly b...,"angry, aggressive, hopeless, anxious, provocat...",8.118,484
3,Predator: Killer of Killers,2025-06-05,movie,85.0,NaN,NaN,"Animation, Action, Science Fiction",Dan Trachtenberg,"Lindsay LaVanchy, Louis Ozawa, Rick Gonzalez, ...",US,en,"20th Century Studios, Davis Entertainment, Law...",While three of the fiercest warriors in human ...,"world war ii, pilot, vikings (norsemen), antho...",7.981,525
4,Lilo & Stitch,2025-05-17,movie,108.0,NaN,NaN,"Family, Science Fiction, Comedy, Adventure",Dean Fleischer Camp,"Maia Kealoha, Sydney Agudong, Chris Sanders, Z...",US,en,"Walt Disney Pictures, Rideback",The wildly funny and touching story of a lonel...,"hawaii, bullying, dysfunctional family, loss o...",7.086,650


In [3]:
df_2025['release_date'] = pd.to_datetime(df_2025['release_date'])
filtered_df = df_2025[
    df_2025['release_date'].between('2025-06-01', '2025-12-31')
]
filtered_df.head()

,title,release_date,type,runtime,season_count,episode_count,genre,director,cast,country,language,production_company,synopsis,tags,imdb_rating,imdb_review_count
2,STRAW,2025-06-05,movie,105.0,NaN,NaN,"Thriller, Drama, Crime",Tyler Perry,"Taraji P. Henson, Sherri Shepherd, Teyana Tayl...",US,en,Tyler Perry Studios,What will be her last straw? A devastatingly b...,"angry, aggressive, hopeless, anxious, provocat...",8.118,484
3,Predator: Killer of Killers,2025-06-05,movie,85.0,NaN,NaN,"Animation, Action, Science Fiction",Dan Trachtenberg,"Lindsay LaVanchy, Louis Ozawa, Rick Gonzalez, ...",US,en,"20th Century Studios, Davis Entertainment, Law...",While three of the fiercest warriors in human ...,"world war ii, pilot, vikings (norsemen), antho...",7.981,525
6,How to Train Your Dragon,2025-06-06,movie,125.0,NaN,NaN,"Action, Family, Fantasy",Dean DeBlois,"Mason Thames, Nico Parker, Gerard Butler, Nick...",US,en,"DreamWorks Animation, Marc Platt Productions","On the rugged isle of Berk, where Vikings and ...","flying, vikings (norsemen), remake, dragon, ba...",8.000,256
7,Deep Cover,2025-06-12,movie,99.0,NaN,NaN,"Action, Comedy, Crime",Tom Kingsley,"Bryce Dallas Howard, Orlando Bloom, Nick Moham...",US,en,"Metronome Film Company, Parkes+MacDonald Image...",Kat is an improv comedy teacher beginning to q...,"london, england, sting operation, met police, ...",6.811,119
9,K.O.,2025-06-05,movie,84.0,NaN,NaN,"Action, Drama, Adventure",Antoine Blossier,"Ciryl Gane, Alice Belaïdi, Maleaume Paquin, Fo...",FR,fr,"Empreinte Digitale, Artémis Productions",A former fighter must find the missing son of ...,NaN,6.660,119


In [4]:
df_2025 = filtered_df
df_2025.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3327 entries, 2 to 15029
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   title               3327 non-null   object        
 1   release_date        3327 non-null   datetime64[ns]
 2   type                3327 non-null   object        
 3   runtime             1563 non-null   float64       
 4   season_count        708 non-null    float64       
 5   episode_count       701 non-null    float64       
 6   genre               2859 non-null   object        
 7   director            2678 non-null   object        
 8   cast                2764 non-null   object        
 9   country             3313 non-null   object        
 10  language            3327 non-null   object        
 11  production_company  2219 non-null   object        
 12  synopsis            2705 non-null   object        
 13  tags                981 non-null    object        
 

In [5]:
df_2025.sort_values(by='title', ascending=True).head(10)

,title,release_date,type,runtime,season_count,episode_count,genre,director,cast,country,language,production_company,synopsis,tags,imdb_rating,imdb_review_count
8786,"""I Don't Wanna Do This Anymore""",2025-09-05,movie,55.0,NaN,NaN,"Horror, Thriller, Drama, Romance",Jessie Crandall,"Tori Daniels, Skyrah Jade, Taylor Gantz, Lando...",US,en,"Vacancy Visions Productions, Scream Soda Inc.,...",Reese finds herself in the desert after a part...,"time loop, slasher, lgbt, lesbian, indie horror",0.0,0
12731,#IBelieveHer,2025-12-31,tv,NaN,1.0,1.0,NaN,NaN,NaN,IE,en,"Pembridge Film Productions, Iris Productions, ...",NaN,NaN,0.0,0
5911,#SalveRosa,2025-10-30,movie,NaN,NaN,NaN,Thriller,Susanna Lira,"Klara Castanho, Karine Teles, Indira Nascimento",BR,pt,"Panorâmica, Elo Studios, Paramount Pictures Br...",A young influencer decides to investigate her ...,NaN,0.0,0
5059,#SchwarzeSchafe,2025-07-17,movie,93.0,NaN,NaN,Comedy,Oliver Rihs,"Jella Haase, Yasin el Harrouk, Jule Böwe, Fred...",DE,de,Port au Prince Films,A Neukölln clan boss becomes a climate activis...,NaN,0.0,0
5056,#SchwarzeSchafe,2025-07-17,movie,93.0,NaN,NaN,Comedy,Oliver Rihs,"Jella Haase, Yasin el Harrouk, Jule Böwe, Fred...",DE,de,Port au Prince Films,A Neukölln clan boss becomes a climate activis...,NaN,0.0,0
2861,& After,2025-11-06,movie,18.0,NaN,NaN,Drama,Alex Holman,"Abby Null, Esmé Chacon, Alex Holman, Matthew M...",US,en,NaN,"On the eve of a major life change, Abby sorts ...","high school, college, melancholy, female frien...",0.0,0
8874,'Lightspeed',2025-07-22,movie,29.0,NaN,NaN,Drama,Robert Lightspeed,"Robert Lightspeed, Nikita Dneprov, Arthur T. H...",US,en,Lightspeed Odysseys,"In a quiet garden where time feels suspended, ...","time, coming of age, existentialism, philosoph...",0.0,0
5126,(Love Song),2025-10-31,movie,NaN,NaN,NaN,NaN,Weerachit Thongjila,"Win Morisaki, Koji Mukai","JP, TH",ja,NaN,An unfinished love song connects two people in...,NaN,0.0,0
3570,(M)OTHER,2025-06-24,movie,NaN,NaN,NaN,Drama,Chi-Min Tsai,"Chin Tzu-yen, Ning Zuo, Chen Yu-hsuan",TW,zh,NaN,"Today is A-Wei’s last day, but it’s her first ...",NaN,0.0,0
8486,(Sur)vivants,2025-06-04,movie,NaN,NaN,NaN,Documentary,Claire Lajeunie,NaN,FR,fr,L2 Films,NaN,women fight,6.5,2


In [6]:
df_2025.isnull().sum()

title                    0
release_date             0
type                     0
runtime               1764
season_count          2619
episode_count         2626
genre                  468
director               649
cast                   563
country                 14
language                 0
production_company    1108
synopsis               622
tags                  2346
imdb_rating              0
imdb_review_count        0
dtype: int64

# 2. 전처리 전처리_type

In [7]:
df_2025[df_2025['season_count'].isna() & df_2025['episode_count'].isna()].head(5)

,title,release_date,type,runtime,season_count,episode_count,genre,director,cast,country,language,production_company,synopsis,tags,imdb_rating,imdb_review_count
2,STRAW,2025-06-05,movie,105.0,NaN,NaN,"Thriller, Drama, Crime",Tyler Perry,"Taraji P. Henson, Sherri Shepherd, Teyana Tayl...",US,en,Tyler Perry Studios,What will be her last straw? A devastatingly b...,"angry, aggressive, hopeless, anxious, provocat...",8.118,484
3,Predator: Killer of Killers,2025-06-05,movie,85.0,NaN,NaN,"Animation, Action, Science Fiction",Dan Trachtenberg,"Lindsay LaVanchy, Louis Ozawa, Rick Gonzalez, ...",US,en,"20th Century Studios, Davis Entertainment, Law...",While three of the fiercest warriors in human ...,"world war ii, pilot, vikings (norsemen), antho...",7.981,525
6,How to Train Your Dragon,2025-06-06,movie,125.0,NaN,NaN,"Action, Family, Fantasy",Dean DeBlois,"Mason Thames, Nico Parker, Gerard Butler, Nick...",US,en,"DreamWorks Animation, Marc Platt Productions","On the rugged isle of Berk, where Vikings and ...","flying, vikings (norsemen), remake, dragon, ba...",8.000,256
7,Deep Cover,2025-06-12,movie,99.0,NaN,NaN,"Action, Comedy, Crime",Tom Kingsley,"Bryce Dallas Howard, Orlando Bloom, Nick Moham...",US,en,"Metronome Film Company, Parkes+MacDonald Image...",Kat is an improv comedy teacher beginning to q...,"london, england, sting operation, met police, ...",6.811,119
9,K.O.,2025-06-05,movie,84.0,NaN,NaN,"Action, Drama, Adventure",Antoine Blossier,"Ciryl Gane, Alice Belaïdi, Maleaume Paquin, Fo...",FR,fr,"Empreinte Digitale, Artémis Productions",A former fighter must find the missing son of ...,NaN,6.660,119


In [8]:
df_2025[df_2025['season_count'].isna() & df_2025['episode_count'].isna()].shape

(2619, 16)

In [9]:
df_2025['type'].value_counts(dropna=False)

type
movie    2619
tv        708
Name: count, dtype: int64

In [10]:
print(df_2025[df_2025['type'] == 'movie']['season_count'].value_counts(dropna=False))
print(df_2025[df_2025['type'] == 'movie']['episode_count'].value_counts(dropna=False))

season_count
NaN    2619
Name: count, dtype: int64
episode_count
NaN    2619
Name: count, dtype: int64


In [11]:
print(df_2025[df_2025['type'] == 'tv']['season_count'].value_counts(dropna=False))

season_count
1.0    692
0.0      7
2.0      6
3.0      2
5.0      1
Name: count, dtype: int64


In [12]:
print(df_2025[df_2025['type'] == 'tv']['episode_count'].value_counts(dropna=False))

episode_count
1.0     334
6.0      59
8.0      46
2.0      41
3.0      39
4.0      37
10.0     23
5.0      22
12.0     20
24.0     14
9.0      12
7.0       9
NaN       7
18.0      4
0.0       4
16.0      4
13.0      3
20.0      3
15.0      3
26.0      3
21.0      2
23.0      2
32.0      2
40.0      2
29.0      1
11.0      1
35.0      1
30.0      1
63.0      1
45.0      1
36.0      1
47.0      1
41.0      1
56.0      1
75.0      1
43.0      1
17.0      1
Name: count, dtype: int64


In [13]:
df_2025_1 = df_2025

In [14]:
# movie인 경우 season/episode count가 NaN이면 1로 채움
df_2025_1.loc[df_2025_1['type'] == 'movie', ['season_count', 'episode_count']] = df_2025_1.loc[df_2025_1['type'] == 'movie', ['season_count', 'episode_count']].fillna(1)
pd.set_option('display.max_columns', None)
df_2025_1.head()

,title,release_date,type,runtime,season_count,episode_count,genre,director,cast,country,language,production_company,synopsis,tags,imdb_rating,imdb_review_count
2,STRAW,2025-06-05,movie,105.0,1.0,1.0,"Thriller, Drama, Crime",Tyler Perry,"Taraji P. Henson, Sherri Shepherd, Teyana Tayl...",US,en,Tyler Perry Studios,What will be her last straw? A devastatingly b...,"angry, aggressive, hopeless, anxious, provocat...",8.118,484
3,Predator: Killer of Killers,2025-06-05,movie,85.0,1.0,1.0,"Animation, Action, Science Fiction",Dan Trachtenberg,"Lindsay LaVanchy, Louis Ozawa, Rick Gonzalez, ...",US,en,"20th Century Studios, Davis Entertainment, Law...",While three of the fiercest warriors in human ...,"world war ii, pilot, vikings (norsemen), antho...",7.981,525
6,How to Train Your Dragon,2025-06-06,movie,125.0,1.0,1.0,"Action, Family, Fantasy",Dean DeBlois,"Mason Thames, Nico Parker, Gerard Butler, Nick...",US,en,"DreamWorks Animation, Marc Platt Productions","On the rugged isle of Berk, where Vikings and ...","flying, vikings (norsemen), remake, dragon, ba...",8.000,256
7,Deep Cover,2025-06-12,movie,99.0,1.0,1.0,"Action, Comedy, Crime",Tom Kingsley,"Bryce Dallas Howard, Orlando Bloom, Nick Moham...",US,en,"Metronome Film Company, Parkes+MacDonald Image...",Kat is an improv comedy teacher beginning to q...,"london, england, sting operation, met police, ...",6.811,119
9,K.O.,2025-06-05,movie,84.0,1.0,1.0,"Action, Drama, Adventure",Antoine Blossier,"Ciryl Gane, Alice Belaïdi, Maleaume Paquin, Fo...",FR,fr,"Empreinte Digitale, Artémis Productions",A former fighter must find the missing son of ...,NaN,6.660,119


In [15]:
df_2025[(df_2025['type'] == 'tv') & (df_2025['season_count'].isna() | df_2025['episode_count'].isna())]
#df_2025[(df_2025['type'] == 'movie') & (df_2025['season_count'].isna() | df_2025['episode_count'].isna())]

,title,release_date,type,runtime,season_count,episode_count,genre,director,cast,country,language,production_company,synopsis,tags,imdb_rating,imdb_review_count
14939,Transaction,2025-06-24,tv,NaN,0.0,NaN,NaN,NaN,NaN,US,en,NaN,NaN,NaN,0.0,0
14940,Noel Edmonds' Kiwi Adventure,2025-06-20,tv,NaN,0.0,NaN,NaN,NaN,NaN,US,en,NaN,NaN,NaN,0.0,0
14941,To Catch a Stalker,2025-07-01,tv,NaN,0.0,NaN,NaN,NaN,NaN,US,en,NaN,NaN,NaN,0.0,0
14942,El negre té nom,2025-06-17,tv,NaN,0.0,NaN,NaN,NaN,NaN,ES,ca,NaN,NaN,NaN,0.0,0
14943,Scrapyard Kings,2025-06-15,tv,NaN,0.0,NaN,NaN,NaN,NaN,US,en,NaN,NaN,NaN,0.0,0
14944,Dream Beyond,2025-06-20,tv,NaN,0.0,NaN,NaN,NaN,NaN,US,en,NaN,NaN,NaN,0.0,0
14945,Fugas Increíbles,2025-06-10,tv,NaN,0.0,NaN,NaN,NaN,NaN,ES,es,NaN,NaN,NaN,0.0,0


In [16]:
print(df_2025_1[df_2025_1['season_count'] == 1.0]['episode_count'].value_counts(dropna=False))

episode_count
1.0     2953
6.0       58
8.0       46
2.0       41
3.0       38
4.0       36
10.0      23
5.0       22
12.0      20
24.0      12
9.0       12
7.0        9
18.0       4
0.0        4
16.0       4
13.0       3
20.0       3
21.0       2
23.0       2
26.0       2
40.0       2
15.0       2
32.0       2
30.0       1
35.0       1
17.0       1
36.0       1
11.0       1
45.0       1
47.0       1
41.0       1
56.0       1
75.0       1
43.0       1
Name: count, dtype: int64


In [17]:
print(df_2025_1[df_2025_1['episode_count'] == 1.0]['season_count'].value_counts(dropna=False))

season_count
1.0    2953
Name: count, dtype: int64


In [18]:
df_2025_2 = df_2025_1
df_2025_2.loc[df_2025_2['type'] == 'tv', ['season_count', 'episode_count']] = df_2025_2.loc[df_2025_1['type'] == 'tv', ['season_count', 'episode_count']].fillna(1)
pd.set_option('display.max_columns', None)
df_2025_2.head()

,title,release_date,type,runtime,season_count,episode_count,genre,director,cast,country,language,production_company,synopsis,tags,imdb_rating,imdb_review_count
2,STRAW,2025-06-05,movie,105.0,1.0,1.0,"Thriller, Drama, Crime",Tyler Perry,"Taraji P. Henson, Sherri Shepherd, Teyana Tayl...",US,en,Tyler Perry Studios,What will be her last straw? A devastatingly b...,"angry, aggressive, hopeless, anxious, provocat...",8.118,484
3,Predator: Killer of Killers,2025-06-05,movie,85.0,1.0,1.0,"Animation, Action, Science Fiction",Dan Trachtenberg,"Lindsay LaVanchy, Louis Ozawa, Rick Gonzalez, ...",US,en,"20th Century Studios, Davis Entertainment, Law...",While three of the fiercest warriors in human ...,"world war ii, pilot, vikings (norsemen), antho...",7.981,525
6,How to Train Your Dragon,2025-06-06,movie,125.0,1.0,1.0,"Action, Family, Fantasy",Dean DeBlois,"Mason Thames, Nico Parker, Gerard Butler, Nick...",US,en,"DreamWorks Animation, Marc Platt Productions","On the rugged isle of Berk, where Vikings and ...","flying, vikings (norsemen), remake, dragon, ba...",8.000,256
7,Deep Cover,2025-06-12,movie,99.0,1.0,1.0,"Action, Comedy, Crime",Tom Kingsley,"Bryce Dallas Howard, Orlando Bloom, Nick Moham...",US,en,"Metronome Film Company, Parkes+MacDonald Image...",Kat is an improv comedy teacher beginning to q...,"london, england, sting operation, met police, ...",6.811,119
9,K.O.,2025-06-05,movie,84.0,1.0,1.0,"Action, Drama, Adventure",Antoine Blossier,"Ciryl Gane, Alice Belaïdi, Maleaume Paquin, Fo...",FR,fr,"Empreinte Digitale, Artémis Productions",A former fighter must find the missing son of ...,NaN,6.660,119


# 2. 데이터 전처리_runtime

In [19]:
df_2025_2.isnull().sum()

title                    0
release_date             0
type                     0
runtime               1764
season_count             0
episode_count            0
genre                  468
director               649
cast                   563
country                 14
language                 0
production_company    1108
synopsis               622
tags                  2346
imdb_rating              0
imdb_review_count        0
dtype: int64

In [20]:
df_2025_2.shape

(3327, 16)

In [21]:
df_2025_2[df_2025_2['runtime'].isna() & df_2025_2['genre'].isna() ].shape

(349, 16)

## 2_2 runtime null 값 열 제거

In [22]:
df_2025_3 = df_2025_2

In [23]:
df_2025_3.dropna(subset=['runtime'], inplace=True)

In [24]:
df_2025_3.shape

(1563, 16)

# 2. 데이터 전처리_pr

In [25]:
df_2025_4 = df_2025_3
df_2025_4

,title,release_date,type,runtime,season_count,episode_count,genre,director,cast,country,language,production_company,synopsis,tags,imdb_rating,imdb_review_count
2,STRAW,2025-06-05,movie,105.0,1.0,1.0,"Thriller, Drama, Crime",Tyler Perry,"Taraji P. Henson, Sherri Shepherd, Teyana Tayl...",US,en,Tyler Perry Studios,What will be her last straw? A devastatingly b...,"angry, aggressive, hopeless, anxious, provocat...",8.118,484
3,Predator: Killer of Killers,2025-06-05,movie,85.0,1.0,1.0,"Animation, Action, Science Fiction",Dan Trachtenberg,"Lindsay LaVanchy, Louis Ozawa, Rick Gonzalez, ...",US,en,"20th Century Studios, Davis Entertainment, Law...",While three of the fiercest warriors in human ...,"world war ii, pilot, vikings (norsemen), antho...",7.981,525
6,How to Train Your Dragon,2025-06-06,movie,125.0,1.0,1.0,"Action, Family, Fantasy",Dean DeBlois,"Mason Thames, Nico Parker, Gerard Butler, Nick...",US,en,"DreamWorks Animation, Marc Platt Productions","On the rugged isle of Berk, where Vikings and ...","flying, vikings (norsemen), remake, dragon, ba...",8.000,256
7,Deep Cover,2025-06-12,movie,99.0,1.0,1.0,"Action, Comedy, Crime",Tom Kingsley,"Bryce Dallas Howard, Orlando Bloom, Nick Moham...",US,en,"Metronome Film Company, Parkes+MacDonald Image...",Kat is an improv comedy teacher beginning to q...,"london, england, sting operation, met police, ...",6.811,119
9,K.O.,2025-06-05,movie,84.0,1.0,1.0,"Action, Drama, Adventure",Antoine Blossier,"Ciryl Gane, Alice Belaïdi, Maleaume Paquin, Fo...",FR,fr,"Empreinte Digitale, Artémis Productions",A former fighter must find the missing son of ...,NaN,6.660,119
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14449,PimpoPlays: Desconectado,2025-06-06,tv,3.0,1.0,1.0,"Action & Adventure, Animation",NaN,NaN,BR,pt,NaN,NaN,NaN,0.000,0
14507,T Déco,2025-06-07,tv,30.0,1.0,1.0,"Reality, Documentary",NaN,NaN,FR,fr,NaN,NaN,NaN,0.000,0
14598,Tu Aashiikii Haii,2025-06-06,tv,20.0,1.0,1.0,Drama,NaN,"Amandeep Sidhu, Abishek Kumar",IN,hi,Dreamiyata Entertainment PVT LTD,Pumma & Noor live out a love that feels incred...,NaN,0.000,0
14679,Après OD - Mexique,2025-06-17,tv,14.0,1.0,1.0,Reality,NaN,NaN,CA,fr,NaN,NaN,NaN,0.000,0


In [26]:
df_2025_4.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1563 entries, 2 to 14740
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   title               1563 non-null   object        
 1   release_date        1563 non-null   datetime64[ns]
 2   type                1563 non-null   object        
 3   runtime             1563 non-null   float64       
 4   season_count        1563 non-null   float64       
 5   episode_count       1563 non-null   float64       
 6   genre               1444 non-null   object        
 7   director            1472 non-null   object        
 8   cast                1360 non-null   object        
 9   country             1561 non-null   object        
 10  language            1563 non-null   object        
 11  production_company  1069 non-null   object        
 12  synopsis            1530 non-null   object        
 13  tags                585 non-null    object        
 

### tags 관련 기본 문제 해결

1) synopsis, tags, genre가 모두 NaN인 데이터 모두 제거

In [27]:
# 1. 세기: synopsis, genre가 모두 NaN인 행 개수
nan_rows = df_2025_4[['synopsis', 'genre']].isna().all(axis=1)
count_nan_rows = nan_rows.sum()
print(f"synopsis, genre가 모두 NaN인 행 개수: {count_nan_rows}")

synopsis, genre가 모두 NaN인 행 개수: 1


In [28]:
# 2. 삭제: 그런 행 삭제
df_2025_4_cleaned = df_2025_4.loc[~nan_rows].copy()

In [29]:
df_2025_4 = df_2025_4.loc[~nan_rows].copy()
df_2025_4.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1562 entries, 2 to 14740
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   title               1562 non-null   object        
 1   release_date        1562 non-null   datetime64[ns]
 2   type                1562 non-null   object        
 3   runtime             1562 non-null   float64       
 4   season_count        1562 non-null   float64       
 5   episode_count       1562 non-null   float64       
 6   genre               1444 non-null   object        
 7   director            1471 non-null   object        
 8   cast                1359 non-null   object        
 9   country             1560 non-null   object        
 10  language            1562 non-null   object        
 11  production_company  1069 non-null   object        
 12  synopsis            1530 non-null   object        
 13  tags                585 non-null    object        
 

In [30]:
df_2025_4

,title,release_date,type,runtime,season_count,episode_count,genre,director,cast,country,language,production_company,synopsis,tags,imdb_rating,imdb_review_count
2,STRAW,2025-06-05,movie,105.0,1.0,1.0,"Thriller, Drama, Crime",Tyler Perry,"Taraji P. Henson, Sherri Shepherd, Teyana Tayl...",US,en,Tyler Perry Studios,What will be her last straw? A devastatingly b...,"angry, aggressive, hopeless, anxious, provocat...",8.118,484
3,Predator: Killer of Killers,2025-06-05,movie,85.0,1.0,1.0,"Animation, Action, Science Fiction",Dan Trachtenberg,"Lindsay LaVanchy, Louis Ozawa, Rick Gonzalez, ...",US,en,"20th Century Studios, Davis Entertainment, Law...",While three of the fiercest warriors in human ...,"world war ii, pilot, vikings (norsemen), antho...",7.981,525
6,How to Train Your Dragon,2025-06-06,movie,125.0,1.0,1.0,"Action, Family, Fantasy",Dean DeBlois,"Mason Thames, Nico Parker, Gerard Butler, Nick...",US,en,"DreamWorks Animation, Marc Platt Productions","On the rugged isle of Berk, where Vikings and ...","flying, vikings (norsemen), remake, dragon, ba...",8.000,256
7,Deep Cover,2025-06-12,movie,99.0,1.0,1.0,"Action, Comedy, Crime",Tom Kingsley,"Bryce Dallas Howard, Orlando Bloom, Nick Moham...",US,en,"Metronome Film Company, Parkes+MacDonald Image...",Kat is an improv comedy teacher beginning to q...,"london, england, sting operation, met police, ...",6.811,119
9,K.O.,2025-06-05,movie,84.0,1.0,1.0,"Action, Drama, Adventure",Antoine Blossier,"Ciryl Gane, Alice Belaïdi, Maleaume Paquin, Fo...",FR,fr,"Empreinte Digitale, Artémis Productions",A former fighter must find the missing son of ...,NaN,6.660,119
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14449,PimpoPlays: Desconectado,2025-06-06,tv,3.0,1.0,1.0,"Action & Adventure, Animation",NaN,NaN,BR,pt,NaN,NaN,NaN,0.000,0
14507,T Déco,2025-06-07,tv,30.0,1.0,1.0,"Reality, Documentary",NaN,NaN,FR,fr,NaN,NaN,NaN,0.000,0
14598,Tu Aashiikii Haii,2025-06-06,tv,20.0,1.0,1.0,Drama,NaN,"Amandeep Sidhu, Abishek Kumar",IN,hi,Dreamiyata Entertainment PVT LTD,Pumma & Noor live out a love that feels incred...,NaN,0.000,0
14679,Après OD - Mexique,2025-06-17,tv,14.0,1.0,1.0,Reality,NaN,NaN,CA,fr,NaN,NaN,NaN,0.000,0


## director와 cast가 모두 nan인 행 제거

In [31]:
nan_both = df_2025_4[(df_2025_4['director'].isna()) & (df_2025_4['cast'].isna())]
print(f"director와 cast 둘 다 NaN인 행 개수: {len(nan_both)}")

director와 cast 둘 다 NaN인 행 개수: 19


In [32]:
df_2025_4 = df_2025_4.dropna(subset=['director', 'cast'], how='all')

In [33]:
df_2025_4.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1543 entries, 2 to 14740
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   title               1543 non-null   object        
 1   release_date        1543 non-null   datetime64[ns]
 2   type                1543 non-null   object        
 3   runtime             1543 non-null   float64       
 4   season_count        1543 non-null   float64       
 5   episode_count       1543 non-null   float64       
 6   genre               1430 non-null   object        
 7   director            1471 non-null   object        
 8   cast                1359 non-null   object        
 9   country             1541 non-null   object        
 10  language            1543 non-null   object        
 11  production_company  1062 non-null   object        
 12  synopsis            1514 non-null   object        
 13  tags                582 non-null    object        
 

# tag npl처리

In [34]:
# 1. tags 결측치는 빈 문자열로 대체
df_2025_4['tags'] = df_2025_4['tags'].fillna('')

# 2. synopsis, genre 결측치 처리 후 text_input 생성
df_2025_4['synopsis'] = df_2025_4['synopsis'].fillna('')
df_2025_4['genre'] = df_2025_4['genre'].fillna('')
df_2025_4['text_input'] = df_2025_4['synopsis'] + ' ' + df_2025_4['genre']

# 3. tags 기준으로 train / test 나누기
train_df = df_2025_4[df_2025_4['tags'] != ''].copy()
test_df = df_2025_4[df_2025_4['tags'] == ''].copy()

# 4. tags 리스트로 분리
train_df['tags_list'] = train_df['tags'].apply(lambda x: x.split(','))

# 5. 다중 라벨 인코딩
mlb = MultiLabelBinarizer()
y_train = mlb.fit_transform(train_df['tags_list'])

# 6. 텍스트 벡터화 (TF-IDF)
vectorizer = TfidfVectorizer(max_features=5000)
X_train = vectorizer.fit_transform(train_df['text_input'])

# 7. 모델 학습 (LogisticRegression 다중 라벨)
model = OneVsRestClassifier(LogisticRegression(max_iter=1000))
model.fit(X_train, y_train)

# 8. 예측 대상(test_df)이 비어있지 않으면 예측 수행
if not test_df.empty:
    X_test = vectorizer.transform(test_df['text_input'])
    y_prob = model.predict_proba(X_test)

    # 예측 임계값 설정 (낮춰서 태그 생성 늘림)
    threshold = 0.07
    y_pred = (y_prob >= threshold).astype(int)

    predicted_tags = mlb.inverse_transform(y_pred)
    test_df['predicted_tags'] = [', '.join(tags) if tags else np.nan for tags in predicted_tags]

    # 9. 규칙 기반 태그 보완용 키워드 룰 정의
    rules = {
        'romance': ['romance', 'love', 'relationship', 'affair', 'heart', 'kiss', 'wedding',
                    'boyfriend', 'girlfriend', 'breakup', 'dating', 'first love',
                    'valentine', 'romantic', 'jealousy', 'intimacy', 'honeymoon'],
        'action': ['battle', 'war', 'fight', 'soldier', 'gun', 'explosion', 'chase', 'enemy',
                'car chase', 'explosives', 'gunfight', 'hand-to-hand', 'sniper',
                'ambush', 'mercenary', 'adrenaline', 'combat', 'high-octane'],
        'thriller': ['thriller', 'mystery', 'suspense', 'crime', 'killer', 'murder', 'detective',
                    'interrogation', 'conspiracy', 'mind game', 'tension', 'abduction',
                    'dangerous', 'mind-bending', 'ticking clock', 'trap'],
        'comedy': ['comedy', 'funny', 'humor', 'laugh', 'joke', 'parody',
                'sitcom', 'sarcasm', 'one-liner', 'slapstick', 'banter',
                'awkward moment', 'gag', 'prank', 'hilarity'],
        'horror': ['ghost', 'horror', 'zombie', 'demon', 'vampire', 'curse',
                'haunted', 'scream', 'paranormal', 'supernatural', 'bloodcurdling',
                'creepy', 'terrifying', 'evil spirit', 'possessed', 'ritual'],
        'fantasy': ['magic', 'wizard', 'dragon', 'kingdom', 'mythical',
                    'sorcerer', 'enchanted', 'portal', 'spell', 'epic quest',
                    'mystical creature', 'ancient prophecy', 'grimoire'],
        'sci-fi': ['space', 'alien', 'robot', 'future', 'technology', 'sci-fi', 'cyber',
                'cyborg', 'AI uprising', 'time machine', 'spaceship', 'hyperspace',
                'galaxy', 'terraforming', 'futuristic', 'parallel universe', 'bioengineering'],
        'drama': ['family', 'life', 'struggle', 'emotional', 'drama', 'society',
                'inner struggle', 'self-discovery', 'character driven', 'emotional arc',
                'breakdown', 'life lesson', 'interpersonal', 'catharsis'],
        'crime': ['criminal', 'robbery', 'heist', 'mafia', 'gang',
                'heist', 'detective', 'undercover', 'criminal syndicate',
                'drug cartel', 'interrogation room', 'forensics', 'manhunt'],
        'animation': ['animation', 'cartoon', 'drawn', 'animated'],
        'sports': ['sports', 'football', 'basketball', 'coach', 'tournament'],
        'historical': ['king', 'queen', 'dynasty', 'empire', 'history', 'ancient'],
        'adventure': ['journey', 'quest', 'treasure', 'explore', 'island', 'survival'],
        'musical': ['song', 'singing', 'performance', 'dance', 'stage', 'music'],
        'documentary': ['real', 'true story', 'interview', 'archive', 'biography', 'footage'],
        'psychological': ['mind', 'mental', 'insanity', 'delusion', 'subconscious', 'dream'],
        'superhero': ['hero', 'superpower', 'villain', 'save', 'city', 'mask'],
        'noir': ['detective', 'dark', 'cigarette', 'femme fatale', 'shadow', 'monologue'],
        'family': ['parent', 'child', 'home', 'holiday', 'siblings', 'bond'],
        'rom-com': ['love', 'funny', 'awkward', 'date', 'charm', 'happy ending'],
        'military': ['army', 'navy', 'mission', 'rank', 'base', 'operation'],
        'cyberpunk': ['neon', 'hacker', 'corporation', 'dystopia', 'implant', 'surveillance'],
        'post-apocalyptic': ['ruins', 'survivor', 'radiation', 'collapse', 'tribe', 'scavenge'],
        'teen': ['high school', 'prom', 'friendship', 'coming of age', 'crush', 'rebellion'],
        'feel-good': ['inspiring', 'heartwarming', 'uplifting', 'positive', 'hope', 'smile'],
        'dark': ['depressing', 'grim', 'tragic', 'bleak', 'morbid'],
        'female-lead': ['strong woman', 'heroine', 'female protagonist', 'girl power'],
        'twist': ['plot twist', 'unpredictable', 'reveal', 'surprise ending'],
        'based-on-true-story': ['true story', 'biopic', 'real events', 'true events'],
        'underdog': ['struggle', 'rise', 'overcome', 'unlikely hero', 'against all odds'],
        'ensemble': ['many characters', 'multiple leads', 'group dynamic', 'interwoven stories'],
        'nostalgic': ['retro', 'classic', 'throwback', '80s', '90s', 'childhood'],
        'social-issue': ['racism', 'inequality', 'poverty', 'justice', 'activism'],
        'trending-theme': ['AI', 'climate', 'pandemic', 'influencer', 'metaverse'],
        'revenge': ['revenge', 'vengeance', 'payback', 'retaliation', 'score to settle'],
        'detective': ['detective', 'investigation', 'clues', 'case', 'inspector', 'sleuth'],
        'time-travel': ['time travel', 'time loop', 'past and future', 'timeline', 'chronosphere'],
        'courtroom': ['courtroom', 'trial', 'lawyer', 'judge', 'defendant', 'testimony'],
        'survival': ['survival', 'wilderness', 'isolation', 'extreme conditions', 'barely escape'],
        'espionage': ['spy', 'espionage', 'agent', 'undercover', 'intel', 'classified'],
        'campus': ['college', 'campus', 'lecture hall', 'seniors', 'dorm', 'professor'],
        'startup': ['startup', 'entrepreneur', 'founder', 'venture', 'pitching', 'angel investor'],
        'heist': ['heist', 'robbery', 'master plan', 'getaway', 'vault', 'crew'],
        'royalty': ['prince', 'princess', 'royal', 'crown', 'throne', 'palace'],
        'wuxia': ['martial arts', 'swordplay', 'wuxia', 'clan', 'sect', 'inner energy'],
        'idol': ['idol', 'stage', 'fandom', 'trainee', 'debut', 'fan meeting'],
        'political': ['election', 'politics', 'scandal', 'congress', 'corruption', 'regime'],
        'revenge-romance': ['bitter love', 'masked identity', 'burning passion', 'payback with love'],
        'healing-romance': ['gentle romance', 'emotional support', 'quiet relationship', 'warm affection'],
        'coming-of-age': ['self-discovery', 'growing up', 'teenage years', 'puberty',
                        'youth', 'maturity', 'identity', 'high school', 'first time', 'life lessons'],
        'dark-comedy': ['dark humor', 'macabre laugh', 'twisted joke', 'morbidly funny',
                        'ironic death', 'satirical tragedy', 'deadpan', 'black comedy'],
        'dystopian': ['oppressive regime', 'totalitarian', 'no freedom', 'surveillance state',
                    'rebellion', 'social collapse', 'harsh world', 'authoritarian', 'controlled society'],
        'slice-of-life': ['daily routine', 'ordinary moments', 'casual life', 'everyday drama',
                        'subtle emotion', 'mundane beauty', 'life as it is', 'quiet narrative'],
        'found-family': ['bond', 'unrelated family', 'loyal companions', 'chosen family',
                        'support system', 'teamwork', 'squad', 'surrogate parents', 'emotional ties'],
        'road-trip': ['on the road', 'journey across', 'miles', 'car travel', 'hit the road',
                    'cross-country', 'travel companions', 'gas station', 'stopover', 'long drive'],
        'tech-noir': ['grim future', 'tech-savvy criminal', 'neon-lit', 'cyber corruption',
                    'urban decay', 'AI rebellion', 'cyberspace', 'digital heist', 'surveillance overload'],
        'folklore': ['legend', 'traditional tale', 'mythical creature', 'ancient lore',
                    'cultural story', 'spirit', 'oral tradition', 'forest demon', 'village tale']
    }

    def simple_tag_rule(text):
        text = text.lower()
        tags = set()
        for tag, keywords in rules.items():
            if any(word in text for word in keywords):
                tags.add(tag)
        return ', '.join(tags) if tags else np.nan

    # 10. 예측 실패 (nan)인 경우 규칙 기반으로 태그 보완
    no_tag_idx = test_df[test_df['predicted_tags'].isna()].index
    test_df.loc[no_tag_idx, 'predicted_tags'] = test_df.loc[no_tag_idx, 'text_input'].apply(simple_tag_rule)

    # 11. 원본에 태그 삽입
    df_2025_4.loc[test_df.index, 'tags'] = test_df['predicted_tags']

C:\Users\User\AppData\Local\Temp\ipykernel_32156\14771476.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2025_4['tags'] = df_2025_4['tags'].fillna('')
C:\Users\User\AppData\Local\Temp\ipykernel_32156\14771476.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2025_4['synopsis'] = df_2025_4['synopsis'].fillna('')
C:\Users\User\AppData\Local\Temp\ipykernel_32156\14771476.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

In [35]:
df_2025_4['tags'].value_counts(dropna=False)

tags
NaN                                                                               52
drama                                                                             33
comedy                                                                            15
drama, thriller                                                                   12
comedy, drama                                                                      9
                                                                                  ..
dyslexia, dyslexic                                                                 1
sci-fi, superhero, adventure, underdog                                             1
folklore, comedy, drama, musical                                                   1
family, drama, adventure, underdog, historical                                     1
psychological, teen, drama, historical, romance, rom-com, documentary, fantasy     1
Name: count, Length: 1251, dtype: int64

# country에서 nan은 language와 같은 값으로 처리

In [36]:
df_2025_4.isnull().sum()

title                   0
release_date            0
type                    0
runtime                 0
season_count            0
episode_count           0
genre                   0
director               72
cast                  184
country                 2
language                0
production_company    481
synopsis                0
tags                   52
imdb_rating             0
imdb_review_count       0
text_input              0
dtype: int64

In [37]:
df_2025_4['country'].fillna(df_2025_4['language'], inplace=True)

C:\Users\User\AppData\Local\Temp\ipykernel_32156\3392914231.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_2025_4['country'].fillna(df_2025_4['language'], inplace=True)
C:\Users\User\AppData\Local\Temp\ipykernel_32156\3392914231.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2025_4['country'].fillna(df_2025_4['language'], inpla

In [38]:
df_2025_4.isnull().sum()

title                   0
release_date            0
type                    0
runtime                 0
season_count            0
episode_count           0
genre                   0
director               72
cast                  184
country                 0
language                0
production_company    481
synopsis                0
tags                   52
imdb_rating             0
imdb_review_count       0
text_input              0
dtype: int64

In [39]:
df_2025_4['production_company'].value_counts(dropna=False)

production_company
NaN                                                                                   481
Elytra Collective                                                                       7
B22 Films, ITN Studios                                                                  2
SVT                                                                                     2
Passion 5 Productions Inc                                                               2
                                                                                     ... 
if... Productions, nordmedia, ERF / Edgar Reitz Filmproduktion                          1
Récifilms, Versus Production, Haut et Court, Cinémage 19, WY Productions, Playtime      1
Big Things Films                                                                        1
Red Bull Studios, The Red Ceiling, Ciné+                                                1
Kinflix, Mojo films (NG)                                                         

# production_company, director, cast _is_comming 처리
- 결측 값이 많아 이렇게 진행
- 정보 부족이지 하나의 회사명, 이름은 x
- 'unknown'만 삽입했을 경우 정상 값이라고 학습하는 것을 방지 => unknown으로 채우되 값은 없음으로 지정하기 위해 사용

In [40]:
unique_count = df_2025_4['production_company'].nunique(dropna=True)
print(unique_count)

1016


In [41]:
df_2025_4['production_company_is_missing'] = df_2025_4['production_company'].isna().astype(int)
df_2025_4['production_company'] = df_2025_4['production_company'].fillna('unknown')

C:\Users\User\AppData\Local\Temp\ipykernel_32156\3369767619.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2025_4['production_company_is_missing'] = df_2025_4['production_company'].isna().astype(int)
C:\Users\User\AppData\Local\Temp\ipykernel_32156\3369767619.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2025_4['production_company'] = df_2025_4['production_company'].fillna('unknown')


In [42]:
df_2025_4['cast_is_missing'] = df_2025_4['cast'].isna().astype(int)
df_2025_4['cast'] = df_2025_4['cast'].fillna('unknown')

C:\Users\User\AppData\Local\Temp\ipykernel_32156\1600149580.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2025_4['cast_is_missing'] = df_2025_4['cast'].isna().astype(int)
C:\Users\User\AppData\Local\Temp\ipykernel_32156\1600149580.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2025_4['cast'] = df_2025_4['cast'].fillna('unknown')


In [43]:
df_2025_4['director_is_missing'] = df_2025_4['director'].isna().astype(int)
df_2025_4['director'] = df_2025_4['director'].fillna('unknown')

C:\Users\User\AppData\Local\Temp\ipykernel_32156\1019100464.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2025_4['director_is_missing'] = df_2025_4['director'].isna().astype(int)
C:\Users\User\AppData\Local\Temp\ipykernel_32156\1019100464.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2025_4['director'] = df_2025_4['director'].fillna('unknown')


In [44]:
print(df_2025_4.shape)
print(df_2025_4.isnull().sum())

(1543, 20)
title                             0
release_date                      0
type                              0
runtime                           0
season_count                      0
episode_count                     0
genre                             0
director                          0
cast                              0
country                           0
language                          0
production_company                0
synopsis                          0
tags                             52
imdb_rating                       0
imdb_review_count                 0
text_input                        0
production_company_is_missing     0
cast_is_missing                   0
director_is_missing               0
dtype: int64


In [45]:
nan_tags_rows = df_2025_4[df_2025_4['tags'].isna()]
nan_tags_rows

,title,release_date,type,runtime,season_count,episode_count,genre,director,cast,country,language,production_company,synopsis,tags,imdb_rating,imdb_review_count,text_input,production_company_is_missing,cast_is_missing,director_is_missing
1040,Lengua Cemento,2025-06-10,movie,31.0,1.0,1.0,Documentary,unknown,"Julian German, Matias Peghini, Luis Benitez, D...",EC,es,papitas con cuero productions,A simple yet powerful portrait of Ecuador's ur...,NaN,8.0,2,A simple yet powerful portrait of Ecuador's ur...,0,0,1
1710,Futsuu no Kodomo,2025-09-05,movie,96.0,1.0,1.0,,Mipo O,"Tetta Shimada, Yu Aoi, Shunsuke Kazama, Kumi T...",JP,ja,murmur,New Mipo O movie.,NaN,0.0,0,New Mipo O movie.,0,0,0
2229,Llamarse Olimpia,2025-06-11,movie,73.0,1.0,1.0,Documentary,Indira Cato,"Olimpia Coral Melo, Marcela Hernández Oropa, P...",MX,es,unknown,Olimpia Coral Melo never set out to become a f...,NaN,0.0,0,Olimpia Coral Melo never set out to become a f...,1,0,0
2353,Dakar: Race Against the Desert,2025-06-26,movie,86.0,1.0,1.0,Documentary,Jalil Lespert,"Sébastien Loeb, Seth Quintero, Toby Price, Cri...",FR,en,"Red Bull Studios, The Red Ceiling, Ciné+",Drivers prepare and compete to win the notorio...,NaN,0.0,0,Drivers prepare and compete to win the notorio...,0,0,0
2557,UNCLAIMED,2025-06-20,movie,90.0,1.0,1.0,,Biodun Stephen,"Kunle Remi, Timini Egbuson, Elma Mbadiwe, Jaiy...",US,en,unknown,Upcoming Nigerian release,NaN,0.0,0,Upcoming Nigerian release,1,0,0
3006,There Is No Antimemetics Division,2025-06-20,movie,17.0,1.0,1.0,,Adria Lang,"Jasika Nicole, Neil Napier, Mateus Ward, Syd K...",US,en,unknown,Directed by Adria Lang,NaN,0.0,0,Directed by Adria Lang,1,0,0
3108,No Country For Old People; a Nursing Home Exposé,2025-08-01,movie,194.0,1.0,1.0,Documentary,Susie Singer Carter,"Susie Singer Carter, Dr. Allen Power, MD, Alys...",US,en,"Go Girl Media, No Country For Old People LLC",A cautionary tale exposing decades of lethal n...,NaN,0.0,0,A cautionary tale exposing decades of lethal n...,0,0,0
3592,Crackers: Folie à Trois,2025-06-18,movie,145.0,1.0,1.0,,Oliver Walker,"Dylan Chapman, Jonah Ozanne, Oliver Walker, To...",US,en,unknown,The Crackers return for one final outing. Will...,NaN,0.0,0,The Crackers return for one final outing. Will...,1,0,0
3774,Richelle's World Tour,2025-12-19,movie,82.0,1.0,1.0,,Richelle,"Richelle, Nick, Josh, Ho, Roggie",US,en,unknown,"After a scary diagnosis, Richelle decided it w...",NaN,0.0,0,"After a scary diagnosis, Richelle decided it w...",1,0,0
3807,Our Land,2025-06-21,movie,89.0,1.0,1.0,Documentary,Orban Wallace,unknown,GB,en,unknown,"Through organised 'mass trespass' events, the ...",NaN,0.0,0,"Through organised 'mass trespass' events, the ...",1,1,0


In [46]:
df_2025_5 = df_2025_4[df_2025_4['tags'].notna()].copy()
df_2025_5.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1491 entries, 2 to 14598
Data columns (total 20 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   title                          1491 non-null   object        
 1   release_date                   1491 non-null   datetime64[ns]
 2   type                           1491 non-null   object        
 3   runtime                        1491 non-null   float64       
 4   season_count                   1491 non-null   float64       
 5   episode_count                  1491 non-null   float64       
 6   genre                          1491 non-null   object        
 7   director                       1491 non-null   object        
 8   cast                           1491 non-null   object        
 9   country                        1491 non-null   object        
 10  language                       1491 non-null   object        
 11  production_company   

In [47]:
df_2025_5.to_csv('전처리_완료(csv)/2025년도 전처리_set.csv', index=False)